#Note: Move this to the home directory of your TPU if you want to load both LingvoTF and MaxText config

In [2]:
import importlib
#the Lingvot TF experiment
module = importlib.import_module('lg-gpt3.lingvo.tasks.lm.params.lg_gpt3')

2023-10-12 21:39:34.032357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-12 21:39:34.032394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
module

<module 'lg-gpt3.lingvo.tasks.lm.params.lg_gpt3' from '/home/mazumdera/lg-gpt3/lingvo/tasks/lm/params/lg_gpt3.py'>

In [4]:
import lingvo.compat as tf
tf.disable_eager_execution()

sess = tf.Session()

tf_model = module.LgTestModel().Task()
tf_model = tf_model.Instantiate()

2023-10-12 21:39:42.448568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-12 21:39:42.448599: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-12 21:39:42.448630: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t1v-n-8e231585-w-0): /proc/driver/nvidia/version does not exist
2023-10-12 21:39:42.449070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
tf_model

In [ ]:
# download the checkpoint if not already done
#!gsutil -m cp -r  "gs://mazumdera-test-bucket/lingvo/v4-16-LgTestModel-10102023-1/train/*" ~/lg-lingvo-checkpoint/


In [6]:
import os
from pprint import pprint

tf_path = os.path.abspath(
    './lg-lingvo-checkpoint')  # Path to our TensorFlow checkpoint

#load the Lingvo TF checkpoint
reader=tf.train.load_checkpoint(tf_path)

In [7]:
ckpt_shape_map = reader.get_variable_to_shape_map()
ckpt_shape_map

{'transformer/decoder/final_layer_norm/w/scale/var/Adafactor': [512],
 'transformer/decoder/final_layer_norm/w/scale/var': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var/Adafactor': [384],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var': [384,
  512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wk/var/Adafactor_1': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wk/var/Adafactor': [384],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var/Adafactor_1': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_001/dense_relu_dense/w/wo/var': [2048,
  512],
 'transformer/dec_pos_emb/w/embedding/var/Adafactor': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/ln/w/scale/var/Adafactor': [512],
 'transformer/dec_emb/w/embedding/var/Adafactor_1': [50272],
 

In [8]:
#Get all the imports
import jax
import os
import sys

jax.config.update('jax_default_prng_impl', 'unsafe_rbg')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["LIBTPU_INIT_ARGS"] = os.environ.get("LIBTPU_INIT_ARGS","") + " --xla_tpu_spmd_rng_bit_generator_unsafe=true"
print(f"Found {jax.device_count()} devices.")

from typing import Sequence
import datetime
from absl import app
from flax.linen import partitioning as nn_partitioning
from flax import linen as nn
import numpy as np
import optax
from tensorboardX import SummaryWriter

from layers import Transformer
import pyconfig
from input_pipeline import create_data_iterator_with_tokenizer
import max_utils
import checkpointing

import jax.numpy as jnp
from jax import random
from jax.sharding import PartitionSpec as P
from jax.sharding import Mesh

from jax.experimental.compilation_cache import compilation_cache as cc

from cloud_tpu_diagnostics import diagnostic
from cloud_tpu_diagnostics.configuration import debug_configuration
from cloud_tpu_diagnostics.configuration import diagnostic_configuration
from cloud_tpu_diagnostics.configuration import stack_trace_configuration

2023-10-12 21:39:59.283759: I external/xla/xla/pjrt/pjrt_api.cc:96] GetPjrtApi was found for tpu at /home/mazumdera/anaconda3/envs/py310/lib/python3.10/site-packages/libtpu/libtpu.so
2023-10-12 21:39:59.283793: I external/xla/xla/pjrt/pjrt_api.cc:69] PJRT plugin for tpu has PJRT API version 0.12. The framework PJRT API version is 0.12.
2023-10-12 21:39:59.283808: I external/xla/xla/pjrt/pjrt_api.cc:58] PJRT_Api is set for device type tpu
2023-10-12 21:39:59.284182: I external/xla/xla/stream_executor/tpu/tpu_initializer_helper.cc:282] Libtpu path is: /home/mazumdera/anaconda3/envs/py310/lib/python3.10/site-packages/libtpu/libtpu.so
2023-10-12 21:40:01.706859: I external/xla/xla/pjrt/pjrt_c_api_client.cc:95] PjRtCApiClient created.
2023-10-12 21:40:01.720418: I external/xla/xla/pjrt/tfrt_cpu_pjrt_client.cc:462] TfrtCpuClient created.


Found 8 devices.


In [9]:
from flax.training import train_state

In [10]:
base_output_directory="base_output_directory=gs://mazumdera-test-bucket/maxtext/lg/10112023/1"
file_pattern_for_train_data="file_pattern_for_train_data=gs://yejingxin-us-central2/external/lg/dummy-data/train/*.tfrecords"
file_pattern_for_eval_data="file_pattern_for_eval_data=gs://yejingxin-us-central2/external/lg/dummy-data/valid/*tfrecords"

In [87]:
commandline_args = ["dummy", "/home/mazumdera/maxtext/MaxText/configs/base.yml","run_name=1xv3-8", "dcn_data_parallelism=1", "save_period=5","ici_data_parallelism=4","ici_tensor_parallelism=2","steps=20","enable_profiler=true","remat_policy=full","base_emb_dim=512","base_num_heads=64","head_dim=64","vocab_size=50272","base_num_decoder_layers=1","per_device_batch_size=0.5","enable_profiler=true", "base_mlp_dim=2048", file_pattern_for_train_data, file_pattern_for_eval_data, base_output_directory,"dataset_type=\"lg\"","max_predict_length=512"]


In [88]:
pyconfig.initialize(commandline_args)
config = pyconfig.config

In [13]:
checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(
      config.checkpoint_dir,
      config.enable_checkpointing,
      config.async_checkpointing,
      config.save_period,
  )

Creating checkpoint manager...


2023-10-12 21:40:17.082218: I external/xla/xla/pjrt/distributed/service.cc:492] Experimental coordination service is enabled.
2023-10-12 21:40:17.082519: I external/xla/xla/pjrt/distributed/service.cc:524] Jax service listening on 10.128.0.69:35353
2023-10-12 21:40:17.083274: I external/tsl/tsl/distributed_runtime/coordination/coordination_service.cc:551] /job:jax_worker/replica:0/task:0 has connected to coordination service. Incarnation: 17886948029794908882
2023-10-12 21:40:17.083432: I external/tsl/tsl/distributed_runtime/coordination/coordination_service_agent.cc:299] Coordination agent has successfully connected.
2023-10-12 21:40:17.083748: I external/xla/xla/pjrt/distributed/client.cc:508] Connected to distributed JAX controller


Checkpoint manager created!


In [89]:
# Initial PRNG Keys
init_rng, nextrng = random.split(random.PRNGKey(config.init_weights_seed), 2)

In [90]:
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)] (num_devices: 8)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1)]]

 [[TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1)]]

 [[TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0)
   TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]]

 [[TpuDevic

In [91]:
model = Transformer(config, mesh)

In [92]:
#TODO: also compare with optax.adafactor
tx = optax.adamw(
      max_utils.create_learning_rate_schedule(config),
      b1=config.adam_b1,
      b2=config.adam_b2,
      eps=config.adam_eps,
      eps_root=config.adam_eps_root,
      weight_decay=config.adam_weight_decay,
  )

In [93]:
state, state_mesh_annotations = max_utils.setup_initial_state(model, tx, config, init_rng, mesh, checkpoint_manager)


restoring state from this run's directory latest step         0


In [94]:
import functools
import max_utils
init_train_state_partial = functools.partial(max_utils.init_train_state, model, tx,
                                               config)

In [95]:
#get the flax.training.train_state.TrainState
abstract_state = jax.eval_shape(init_train_state_partial, init_rng)
abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fca62bd43a0>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 1, 2048), dtype=float32), names=('embed', 'layers', 'mlp'), mesh=None, rules=None)}, 'wo': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(2048, 1, 512), dtype=float32), names=('mlp', 'layers', 'embed'), mesh=None, rules=None)}}, 'pre_self_attention_layer_norm': {'scale': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 1), dtype=float32), names=('embed', 'layers'), mesh=None, rules=None)}, 'relpos_bias': {'rel_embedding': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(64, 1, 32), dtype=float32), na

In [96]:

state_logical_annotations = nn.get_partition_spec(abstract_state)
state_logical_annotations

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fca62bd43a0>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': PartitionSpec('embed', 'layers', 'mlp')}, 'wo': {'kernel': PartitionSpec('mlp', 'layers', 'embed')}}, 'pre_self_attention_layer_norm': {'scale': PartitionSpec('embed', 'layers')}, 'relpos_bias': {'rel_embedding': PartitionSpec('heads', 'layers', 'relpos_buckets')}, 'self_attention': {'key': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'key_layer_norm': {'scale': PartitionSpec('heads', 'layers')}, 'out': {'kernel': PartitionSpec('heads', 'layers', 'kv', 'embed')}, 'query': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'query_layer_norm': {'scale': PartitionSpec('heads', 'l

In [97]:
unboxed_abstract_state = max_utils.unbox_logicallypartioned_trainstate(abstract_state)
unboxed_abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fca62bd43a0>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': ShapeDtypeStruct(shape=(512, 1, 2048), dtype=float32)}, 'wo': {'kernel': ShapeDtypeStruct(shape=(2048, 1, 512), dtype=float32)}}, 'pre_self_attention_layer_norm': {'scale': ShapeDtypeStruct(shape=(512, 1), dtype=float32)}, 'relpos_bias': {'rel_embedding': ShapeDtypeStruct(shape=(64, 1, 32), dtype=float32)}, 'self_attention': {'key': {'kernel': ShapeDtypeStruct(shape=(512, 1, 64, 64), dtype=float32)}, 'key_layer_norm': {'scale': ShapeDtypeStruct(shape=(64, 1), dtype=float32)}, 'out': {'kernel': ShapeDtypeStruct(shape=(64, 1, 64, 512), dtype=float32)}, 'query': {'kernel': ShapeDty

In [98]:
#Num of params in the MaxText config

from train import calculate_num_params_from_pytree
print(f"Total num of params = {calculate_num_params_from_pytree(state.params)}")

Total num of params = 340142208


In [25]:
#Let us consider we are saving a state at step 0 at the vanilla state
import max_logging
step = 0
if checkpoint_manager is not None:
      if checkpoint_manager.save(step, state):
        max_logging.log(f"saved a checkpoint at step {step}")

In [ ]:
# #since we know know we are loading checkpoint at step 50
# step = 50
# if checkpoint_manager is not None:
#       if checkpoint_manager.save(step, state):
#         max_logging.log(f"saved a checkpoint at step {step}")

In [39]:
# all_keys = [key for key in state.params.keys]
print(state.params['decoder']['decoder'].keys())

dict_keys(['mlp', 'pre_self_attention_layer_norm', 'relpos_bias', 'self_attention'])


In [44]:
state.params['decoder']['decoder'].keys()

dict_keys(['mlp', 'pre_self_attention_layer_norm', 'relpos_bias', 'self_attention'])

In [45]:
state.params['decoder']['decoder'].keys()

dict_keys(['mlp', 'pre_self_attention_layer_norm', 'relpos_bias', 'self_attention'])

In [99]:
state.params['decoder']['decoder']['mlp']['wi']['kernel'].shape

(4096, 1, 8192)